In [1]:
# Loading the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import os
import cv2
from tqdm import tqdm

2025-04-24 01:54:40.116532: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 01:54:40.227402: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-24 01:54:40.306613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745456080.391440   14249 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745456080.416340   14249 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745456080.582321   14249 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 0


E0000 00:00:1745456090.379373   14249 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1745456090.433290   14249 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:

# Loading the dataset
import os
import cv2
from tqdm import tqdm

# Loading the dataset and preparing it for training
train_path = '../dataset_organized/train/'
test_path = '../dataset_organized/test/'
val_path = '../dataset_organized/validation/'

def load_images(path):
    X = []
    y = []
    
    for folder in os.listdir(path):
        files = os.listdir(path + folder)
        for file in tqdm(files):
            img = cv2.imread(path + folder + '/' + file)
            img = cv2.resize(img, (150, 150))
            X.append(img)
            y.append(folder)
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y

X_train, y_train = load_images(train_path)
X_test, y_test = load_images(test_path)
X_val, y_val = load_images(val_path)

classes_names = np.unique(y_train)
num_classes = len(classes_names)
print(f"Number of classes: {num_classes}")
print(f"Class names: {classes_names}")

# Encoding the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_val_encoded = label_encoder.transform(y_val)

# One hot encoding the target variable
y_train = to_categorical(y_train_encoded)
y_test = to_categorical(y_test_encoded)
y_val = to_categorical(y_val_encoded)

# Normalizing the images
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")

# Enhanced Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]
)

datagen.fit(X_train)


100%|██████████| 17/17 [00:00<00:00, 822.02it/s]


Number of classes: 6
Class names: ['cardboard' 'glass' 'metal' 'paper' 'plastic' 'trash']
Training set shape: (1768, 150, 150, 3)
Validation set shape: (328, 150, 150, 3)
Test set shape: (431, 150, 150, 3)


In [ ]:

# Define the hypermodel for DenseNet
def build_model(hp):
    # Select DenseNet variant
    densenet_version = hp.Choice('densenet_version', values=['DenseNet121', 'DenseNet169', 'DenseNet201'])
    
    if densenet_version == 'DenseNet121':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    elif densenet_version == 'DenseNet169':
        base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    else:
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    
    # Freeze base model layers
    freeze_option = hp.Choice('freeze_option', values=['all', 'partial', 'none'])
    
    if freeze_option == 'all':
        for layer in base_model.layers:
            layer.trainable = False
    elif freeze_option == 'partial':
        # Freeze first 75% of the layers
        total_layers = len(base_model.layers)
        for layer in base_model.layers[:int(total_layers * 0.75)]:
            layer.trainable = False
        for layer in base_model.layers[int(total_layers * 0.75):]:
            layer.trainable = True
    else:  # 'none'
        for layer in base_model.layers:
            layer.trainable = True
    
    # Add custom top layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    # Add dense layers
    num_dense_layers = hp.Int('num_dense_layers', min_value=1, max_value=3)
    
    for i in range(num_dense_layers):
        units = hp.Int(f'dense_{i+1}_units', min_value=64, max_value=512, step=64)
        activation = hp.Choice(f'dense_{i+1}_activation', values=['relu', 'tanh', 'elu'])
        
        x = Dense(units=units, activation=activation)(x)
        
        # Add batch normalization option
        use_batch_norm = hp.Boolean(f'batch_norm_{i+1}')
        if use_batch_norm:
            x = BatchNormalization()(x)
            
        dropout_rate = hp.Float(f'dropout_{i+1}_rate', min_value=0.2, max_value=0.6, step=0.1)
        x = Dropout(dropout_rate)(x)
    
    # Output layer
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    if optimizer == 'adam':
        opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    else:
        momentum = hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        opt = SGD(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]), momentum=momentum)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Number of hyperparameter combinations to try
    executions_per_trial=1,  # Number of models to train per combination
    directory='densenet_tuning',
    project_name='densenet_hyperparameter_tuning'
)

# Print search space summary
tuner.search_space_summary()

# Search for the best hyperparameters
tuner.search(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=10,
    validation_data=(X_val, y_val)
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print detailed information about the optimal hyperparameters
densenet_version = best_hps.get('densenet_version')
freeze_option = best_hps.get('freeze_option')
num_dense_layers = best_hps.get('num_dense_layers')

# Prepare information about dense layers
dense_layers_info = ""
for i in range(num_dense_layers):
    units = best_hps.get(f'dense_{i+1}_units')
    activation = best_hps.get(f'dense_{i+1}_activation')
    batch_norm = best_hps.get(f'batch_norm_{i+1}', False)
    dropout_rate = best_hps.get(f'dropout_{i+1}_rate')
    
    dense_layers_info += f"\n- Layer {i+1}: {units} units, activation: {activation}, "
    dense_layers_info += f"batch normalization: {'Yes' if batch_norm else 'No'}, "
    dense_layers_info += f"dropout rate: {dropout_rate}"

# Prepare optimizer information
optimizer = best_hps.get('optimizer')
learning_rate = best_hps.get('learning_rate')
momentum_info = ""
if optimizer == 'sgd':
    momentum = best_hps.get('momentum')
    momentum_info = f", momentum: {momentum}"

print(f"""
The hyperparameter search is complete. The optimal architecture consists of:

Base model: {densenet_version}
Freeze option: {freeze_option}

Dense layers ({num_dense_layers}):{dense_layers_info}

Optimizer: {optimizer} with learning rate: {learning_rate}{momentum_info}
""")

# Build the model with the best hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
model.summary()

# Train the model with the best hyperparameters
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    epochs=20,  # Train for longer than during search
    validation_data=(X_val, y_val)
)

# Save the best model
model.save('hypertuned_densenet_model.h5')

# Plotting the training and validation accuracy and loss side by side
plt.figure(figsize=(15, 5))

# Subplot for accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# Subplot for loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('densenet_training_history.png')
plt.show()

# Evaluating the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

# Get training and validation metrics from history
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]
train_loss = history.history['loss'][-1]
val_loss = history.history['val_loss'][-1]

print('Training Accuracy:', train_acc)
print('Validation Accuracy:', val_acc)
print('Training Loss:', train_loss)
print('Validation Loss:', val_loss)

# Create confusion matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes_names, yticklabels=classes_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.savefig('densenet_confusion_matrix.png')
plt.show()

# Print classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=classes_names))


Trial 7 Complete [00h 18m 02s]
val_accuracy: 0.25

Best val_accuracy So Far: 0.2591463327407837
Total elapsed time: 00h 57m 15s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
DenseNet121       |DenseNet169       |densenet_version
none              |all               |freeze_option
2                 |1                 |num_dense_layers
192               |192               |dense_1_units
elu               |relu              |dense_1_activation
True              |False             |batch_norm_1
0.4               |0.2               |dropout_1_rate
rmsprop           |rmsprop           |optimizer
0.001             |0.001             |learning_rate
384               |448               |dense_2_units
relu              |relu              |dense_2_activation
False             |True              |batch_norm_2
0.2               |0.4               |dropout_2_rate
384               |384               |dense_3_units
relu              |tanh              |dense_3_activa